In [36]:
import requests
import json
files = { "image" : open("Marco.jpg", "rb")}
resp = requests.post("https://asc.habiliseurope.com/face-service/process", files=files)
json.dump(resp.json(), open("embeddings-marco.json", "w"))

In [13]:
import json

emb1 = json.load(open("embeddings.json"))
emb2 = json.load(open("embeddings2.json"))

reqjson = {
    "target" : emb1[0]["embedding"],
    "sources" : [emb["embedding"] for emb in emb2],
    "threshold" : 0.7
}

resp = requests.post("https://asc.habiliseurope.com/face-service/compare", json=reqjson)
resp.json()

[{'match': False, 'similarity': 0.4073483920088131},
 {'match': False, 'similarity': 0.17019717348296282},
 {'match': True, 'similarity': 0.7608417798114866}]

In [34]:
from PIL import Image, ImageDraw, ImageFont

emb2 = json.load(open("embeddings-corso.json"))
image = Image.open("gruppo-corso.jpg")
draw = ImageDraw.Draw(image)
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", size=100)
except IOError:
    font = ImageFont.load_default()

for emb in emb2:
    box = tuple(emb["box"])
    color = "red" if emb["confidence"] < 0.9 else "green"
    draw.rectangle(box, width=3, outline=color)
    confidence = round(100 * emb["confidence"])
    draw.text((box[0], box[3]), str(confidence) + "%", fill="cyan", font=font)

image.save("labelled_gruppo-corso.jpeg")

In [39]:
from PIL import Image, ImageDraw, ImageFont

tgt_embs = json.load(open("embeddings-marco.json"))
src_embs = json.load(open("embeddings-corso.json"))

image = Image.open("gruppo-corso.jpg")
draw = ImageDraw.Draw(image)
try:
    font = ImageFont.truetype("itineris-dataspace/DejaVuSans.ttf", size=25)
except IOError:
    font = ImageFont.load_default()

sources = [ src["embedding"] for src in src_embs ]
    
for tgt in tgt_embs:
    tgt_emb = tgt["embedding"]
    reqjson = {
        "target" : tgt_emb,
        "sources" : sources,
        "threshold" : 0.5
    }
    resp = requests.post("https://asc.habiliseurope.com/face-service/compare", json=reqjson)
    matches = resp.json()
    print(matches)
    for i in range(len(matches)):
        match = matches[i]
        if match["match"]:
            box = tuple(src_embs[i]["box"])
            color = "red" if match["similarity"] < 0.8 else "green"
            draw.rectangle(box, width=3, outline=color)
            similarity = round(100 * match["similarity"])
            draw.text((box[0], box[3]), "Marco (" + str(similarity) + "%)", fill="cyan", font=font)

image.save("compared_gruppo-corso.jpg")

[{'match': False, 'similarity': 0.17992300049751675}, {'match': False, 'similarity': 0.24792434261952556}, {'match': False, 'similarity': -0.06868963943533934}, {'match': False, 'similarity': 0.07500252732328139}, {'match': False, 'similarity': -0.1042637256608443}, {'match': False, 'similarity': 0.1408155130181602}, {'match': True, 'similarity': 0.572850610115463}, {'match': False, 'similarity': 0.07530882049946186}, {'match': False, 'similarity': 0.10150518296249933}, {'match': False, 'similarity': 0.01634554460137838}, {'match': False, 'similarity': 0.3208911363480338}]
